In [ ]:
# Lang Chain movie script exercise


In [34]:
import os
import configparser
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_cohere import ChatCohere
from langchain_groq import ChatGroq
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [35]:
config = configparser.ConfigParser()
config.read('../config.ini')  

groq = config['groq']
cohere = config['cohere']

os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')


In [ ]:
import random

def actor_picker():
    actors = ["Chris Evans", "Tom Hanks", "Robert Downey Jr.", "Shah Rukh Khan", "Amitabh Bachchan", "Ranveer Singh"]
    return random.choice(actors)

def location_picker():
    locations = ["New York", "Paris", "Tokyo", "Mumbai", "Delhi", "Chennai"]
    return random.choice(locations)

def theme_picker():
    themes = ["Adventure", "Mystery", "Romance", "Comedy", "Thriller", "Drama"]
    return random.choice(themes)


In [37]:
cohere_model = ChatCohere(model="command-r-plus")
groq_model = ChatGroq(model="llama3-8b-8192")


In [38]:
def create_script_prompt_with_context(actor, location, theme):
    system_message = SystemMessage(content="You are a professional movie script writer. Your task is to create compelling scripts based on given actors, locations, and themes. Include scene descriptions, character dialogues, and a plot twist.")
    human_message = HumanMessage(content=f"Write a short movie script starring {actor}, set in {location}, with the theme of {theme}. Make it engaging and thrilling.")
    
    return [system_message, human_message]

In [39]:
def story_cohere():
    actor = actor_picker()
    location = location_picker()
    theme = theme_picker()
    
    story_template = story_temp(actor, location, theme)
    response = cohere_model.invoke([HumanMessage(content=story_template)])
    return response.content

def story_groq():
    actor = actor_picker()
    location = location_picker()
    theme = theme_picker()
    
    story_template = story_temp(actor, location, theme)
    response = groq_model.invoke([HumanMessage(content=story_template)])
    return response.content


In [40]:
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


In [41]:
def update_story(user_input, session_id, model_choice):
    session_history = get_session_history(session_id)
    session_history.add_message(HumanMessage(content=user_input))
    
    messages = session_history.messages  
    
    if model_choice == 'cohere':
        response = cohere_model.invoke([message for message in messages if isinstance(message, (HumanMessage, SystemMessage))])
    elif model_choice == 'groq':
        response = groq_model.invoke([message for message in messages if isinstance(message, (HumanMessage, SystemMessage))])
    else:
        return "Invalid model choice"
    
    updated_story = response.content
    session_history.add_message(SystemMessage(content=updated_story))
    
    return updated_story


In [ ]:
session_id = "test"

print("Generating a short story using Cohere:")
cohere_story = story_cohere()
print(f"Story from Cohere Model:\n{cohere_story}\n")
store[session_id] = InMemoryChatMessageHistory()
store[session_id].add_message(SystemMessage(content=cohere_story))
print("Generating a short story using Groq:")
groq_story = story_groq()
print(f"Story from Groq Model:\n{groq_story}\n")
store[session_id].add_message(SystemMessage(content=groq_story))
user_input = input("Enter your update for the story (or type 'exit' to finish): ")
print("\nUpdating story using Cohere...")
updated_story = update_story(user_input, session_id, model_choice='cohere')
print(f"Updated Story (Cohere):\n\n{updated_story}\n")
print("Updating story using Groq...")
updated_story_groq = update_story(user_input, session_id, model_choice='groq')
print(f"Updated Story (Groq):\n\n{updated_story_groq}\n")


Generating a short story using Cohere:
Story from Cohere Model:
Tom Hanks found himself in the bustling city of Los Angeles, eager to embark on a new adventure. With his curious spirit, he explored the vibrant streets of the city, from the bustling Hollywood Boulevard to the serene beaches of Venice. 

As he wandered, he discovered a hidden trailhead, beckoning him to venture into the unknown. He stepped into a world of mystery and excitement, where he encountered unique characters and uncovered the city's hidden gems. 

With each new discovery, Tom embraced the thrill of the unknown, creating memories that would forever be etched in his mind. Thus, his Los Angeles adventure became a testament to the power of exploration and the magic that awaits those who dare to seek it.

Generating a short story using Groq:
Story from Groq Model:
Robert Downey Jr. strolled through Tokyo's neon-lit streets, his Tony Stark wit at the ready. As he navigated a crowded ramen shop, he accidentally knocked

In [44]:
print(store)

{'test': InMemoryChatMessageHistory(messages=[SystemMessage(content="Tom Hanks found himself in the bustling city of Los Angeles, eager to embark on a new adventure. With his curious spirit, he explored the vibrant streets of the city, from the bustling Hollywood Boulevard to the serene beaches of Venice. \n\nAs he wandered, he discovered a hidden trailhead, beckoning him to venture into the unknown. He stepped into a world of mystery and excitement, where he encountered unique characters and uncovered the city's hidden gems. \n\nWith each new discovery, Tom embraced the thrill of the unknown, creating memories that would forever be etched in his mind. Thus, his Los Angeles adventure became a testament to the power of exploration and the magic that awaits those who dare to seek it.", additional_kwargs={}, response_metadata={}), SystemMessage(content='Robert Downey Jr. strolled through Tokyo\'s neon-lit streets, his Tony Stark wit at the ready. As he navigated a crowded ramen shop, he a